In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# GPU를 사용 가능하면 할당.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

Using cuda device


In [26]:
#모델 구현 클래스

class NeuralNetwork(nn.Module):

    def __init__(self):  # 모델을 인스턴스화할때, 딱 한번 실행이 되는데 레이어 층을 구현하는 함수다.
        print("init Callback!")
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    def forward(self, x):
        print("origin x : ",x.shape)
        x = self.flatten(x) # 데이터를 학습 할 수 있게, 1차원으로 reshape 한다.
        print("x : ", x.shape)
        logits = self.linear_relu_stack(x) # 위의 init에 구현된 linear_relu_stack을 활용.
        return logits

In [27]:
# 인스턴스 생성
model = NeuralNetwork().to(device)
print(model)

init Callback!
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [37]:
# 임시 데이터 생성
X = torch.rand(1,28, 28, device=device)

# 만든 모델에 입력
logits = model(X)
print(logits)

# 마지막에 다중분류에 특화된 softmax 맥임.
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

origin x :  torch.Size([1, 28, 28])
x :  torch.Size([1, 784])
tensor([[0.0000, 0.0000, 0.0000, 0.0554, 0.0000, 0.1010, 0.0000, 0.0000, 0.0000,
         0.0401]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.0980, 0.0980, 0.0980, 0.1036, 0.0980, 0.1084, 0.0980, 0.0980, 0.0980,
         0.1020]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class : tensor([5], device='cuda:0')


# 모델 계층(Layer)

In [29]:
# 임시 모델 생성
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [30]:
flatten = nn.Flatten()
flat_image = flatten(input_image) # 학습할 수 있게 reshape
print(flat_image.size())

torch.Size([3, 784])


In [33]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # 레이어 층 만들고
hidden1 = layer1(flat_image) # 데이터 집어넣고 반환.
print(hidden1)
print(hidden1.shape)

tensor([[ 0.2912,  0.5014,  0.0511, -0.1685, -0.4598, -0.1709, -0.1291,  0.2767,
          0.5760,  0.3112, -0.3815, -0.0134, -0.3582,  0.1622, -0.3334, -0.0987,
         -0.1561,  0.2987,  0.4022,  0.0617],
        [ 0.1432,  0.1082,  0.1536, -0.2488, -0.0448,  0.1661, -0.1316,  0.4279,
          0.3025,  0.0688, -0.4521, -0.2629,  0.3072,  0.1598,  0.1345, -0.2644,
         -0.6310,  0.4670,  0.2429,  0.4814],
        [ 0.1228,  0.0988,  0.3831, -0.0244, -0.3794, -0.1114,  0.0551,  0.1470,
          0.4379,  0.1929, -0.2155, -0.0739, -0.2644,  0.1255, -0.1824, -0.1541,
         -0.6471,  0.5988,  0.3571,  0.2197]], grad_fn=<AddmmBackward>)
torch.Size([3, 20])


In [34]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1) # 활성화 함수에 집어넣고 반환
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2912,  0.5014,  0.0511, -0.1685, -0.4598, -0.1709, -0.1291,  0.2767,
          0.5760,  0.3112, -0.3815, -0.0134, -0.3582,  0.1622, -0.3334, -0.0987,
         -0.1561,  0.2987,  0.4022,  0.0617],
        [ 0.1432,  0.1082,  0.1536, -0.2488, -0.0448,  0.1661, -0.1316,  0.4279,
          0.3025,  0.0688, -0.4521, -0.2629,  0.3072,  0.1598,  0.1345, -0.2644,
         -0.6310,  0.4670,  0.2429,  0.4814],
        [ 0.1228,  0.0988,  0.3831, -0.0244, -0.3794, -0.1114,  0.0551,  0.1470,
          0.4379,  0.1929, -0.2155, -0.0739, -0.2644,  0.1255, -0.1824, -0.1541,
         -0.6471,  0.5988,  0.3571,  0.2197]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.2912, 0.5014, 0.0511, 0.0000, 0.0000, 0.0000, 0.0000, 0.2767, 0.5760,
         0.3112, 0.0000, 0.0000, 0.0000, 0.1622, 0.0000, 0.0000, 0.0000, 0.2987,
         0.4022, 0.0617],
        [0.1432, 0.1082, 0.1536, 0.0000, 0.0000, 0.1661, 0.0000, 0.4279, 0.3025,
         0.0688, 0.0000, 0.0000, 0.3072, 0.1598, 0.134

In [ ]:
seq_modules = nn.Sequential( # 위에 한걸 한번에 모아서 가능.
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[-0.1323,  0.1455,  0.0748, -0.2269, -0.0986,  0.0580, -0.0362,  0.3052,
          0.2291, -0.2180],
        [-0.1181,  0.2191,  0.1445, -0.1876, -0.0245,  0.0836, -0.0294,  0.2530,
          0.3107, -0.2355],
        [-0.1029,  0.1735, -0.0111, -0.1858, -0.1057,  0.0999, -0.0563,  0.2871,
          0.2004, -0.2145]], grad_fn=<AddmmBackward>)


In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0854, 0.1128, 0.1051, 0.0777, 0.0884, 0.1033, 0.0940, 0.1323, 0.1226,
         0.0784],
        [0.0839, 0.1175, 0.1091, 0.0782, 0.0921, 0.1026, 0.0917, 0.1216, 0.1288,
         0.0746],
        [0.0883, 0.1164, 0.0967, 0.0812, 0.0880, 0.1081, 0.0925, 0.1304, 0.1195,
         0.0789]], grad_fn=<SoftmaxBackward>)


In [ ]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} \n Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) 
 Values : tensor([[-0.0170, -0.0325,  0.0262,  ...,  0.0299,  0.0084,  0.0099],
        [-0.0294,  0.0196, -0.0114,  ...,  0.0324,  0.0055,  0.0225]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) 
 Values : tensor([ 0.0099, -0.0054], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) 
 Values : tensor([[-0.0010,  0.0025,  0.0155,  ..., -0.0372, -0.0080,  0.0442],
        [ 0.0160, -0.0006, -0.0211,  ..., -0.0343,  0.0344,  0.0206]],
       device='cuda